In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from IPython.display import Markdown

In [3]:
GOOGLE_API_KEY=os.environ['GOOGLE_API_KEY']

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA

/Users/aayushchaudhary/Git/college/literature-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the Paper

`Set proper name of the path to the paper's pdf`

If loading single dcoument: 

In [7]:
loader = PyPDFLoader('./papers/0.pdf')
data = loader.load()

Multiple Docs

`here set the directory where you have the papers`

In [8]:
loader = DirectoryLoader(path='./papers/', glob='**/*.pdf', loader_cls=PyPDFLoader)
data = loader.load()

Data is loaded page by page

In [28]:
data[12].page_content

'S2275 Attention-based graph neural networks: a\xa0survey  \n1 3\nInter-layer GATs: This kind of works usually select features beyond neural network \nlayers with multiple feature spaces, not just local neighborhoods. Across the neural net-\nwork layer, attention in inter-layer GATs can be regarded as an operation of cross-layer \nfusion of different feature spaces with feature fusion attention. In this term, attention-\nbased GNNs dynamically select features from different levels, different channels, dif-\nferent views, or different time slices. Therefore, we further divide these methods into \nfive sub-categories (i.e., multi-level attention (Liu et\xa0 al. 2020; Zhang et\xa0 al. 2022c), \nmulti-channel (Bo et\xa0al. 2021; Luan et\xa0al. 2021), multi-view (Wang et\xa0al. 2020b; Yuan \net\xa0al. 2021b), Spatio-temporal attention (Sankar et\xa0al. 2018; Lu et\xa0al. 2019), and time \nseries attention (Zhang et\xa0al. 2021c; Zhao et\xa0al. 2020)). By considering temporal attrib-\nutes, 

### Split all the docs into Chunks

Split pages into 1000 word chunks with buffer/overlap of 200 words

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [ ]:
texts[0].page_content

Generate model embeddings

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
vector_store = Chroma.from_documents(documents=texts, embedding=embeddings)

I0000 00:00:1722946400.395771   43860 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Setup Retriver

In [11]:
retriever = vector_store.as_retriever()

Setup LLM to be used and chain

In [5]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True
)

A Query to generate review of all papers: 

`Should contain all the names of your papers`

In [29]:
query = ("""
You are a [Research Assitant] bot. You help with creating [Literature Review]. 

Input :You will be given access to a [research paper]


Task: You have to [extract] the following information:
Information To BE extracted is present in backticks: 

Paper Name
Focus Area of the paper
Date: month/year
Methodology: (eg Qualitative, Quantitative, Review, Conceptual, Report)
Key Findings: in 10 words what the paper has implemented/achieved
Application: real life potential use cases (summerize in 10 words or 1-2 points)
Challenges:  Drawbacks of this paper/approach (summerize 1-2 short points)
Opportunities: Future scope/possibilities of paper (summerize in 1-2 points)
Dataset: (dataset used in the paper if available else write none) 

Note: if including multiple points, insert newline character or write a unordered list in markdown wrapped as a string

Output Format is inside triple backticks: 
```
python
[
        
{
"paper_name": "name of the paper 0",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
},
{
"paper_name": "name of the paper 1",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
}
...
]

```
         
Names of the papers: 
<<<
A Survey on Graph Neural Networks for Time
Series: Forecasting, Classification, Imputation,
and Anomaly Detection, 
Attention Is All You Need,
Exploring Hierarchical Structures for Recommender Systems,
Heterogeneous Graph Attention Network, 
Hyperbolic Graph Attention Network, 
KGAT: Knowledge Graph Attention Network for Recommendation, 
Knowledge Graph Embedding Based on Graph Neural Network,
Research on the application of Nerual Network model in knowledge graph completion technology,
Attention based graph neural networks: a survey, 
Graph neural networks for visual question answering: a systematic review, 
A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions
>>>
""")

### Run the chain

In [30]:
result = qa_chain.invoke({'query': query})

Result is Stored in `result` object and can be accessed using key: 

In [31]:
result['result']

'```python\n[\n{\n"paper_name": "A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection",\n"Focus Area of the paper": "Graph Neural Networks for Time Series",\n"Date": "12/2022",\n"Methodology": "Review",\n"Key Findings": "Surveyed GNNs for time series tasks",\n"Application": "Forecasting, classification, imputation, anomaly detection",\n"Challenges": "Limited data availability, model complexity",\n"Opportunities": "Explore new architectures, improve interpretability",\n"datatset": "None"\n},\n{\n"paper_name": "Attention Is All You Need",\n"Focus Area of the paper": "Attention Mechanism in Neural Networks",\n"Date": "06/2017",\n"Methodology": "Quantitative",\n"Key Findings": "Proposed Transformer architecture based on attention",\n"Application": "Machine translation, text summarization",\n"Challenges": "High computational cost, lack of interpretability",\n"Opportunities": "Explore new attention mechanisms, improve efficiency",\

### Our Output is formatted in a way that it can be parsed as a python list of dicts/JSON objects

In [32]:
Markdown(result['result'])

```python
[
{
"paper_name": "A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection",
"Focus Area of the paper": "Graph Neural Networks for Time Series",
"Date": "12/2022",
"Methodology": "Review",
"Key Findings": "Surveyed GNNs for time series tasks",
"Application": "Forecasting, classification, imputation, anomaly detection",
"Challenges": "Limited data availability, model complexity",
"Opportunities": "Explore new architectures, improve interpretability",
"datatset": "None"
},
{
"paper_name": "Attention Is All You Need",
"Focus Area of the paper": "Attention Mechanism in Neural Networks",
"Date": "06/2017",
"Methodology": "Quantitative",
"Key Findings": "Proposed Transformer architecture based on attention",
"Application": "Machine translation, text summarization",
"Challenges": "High computational cost, lack of interpretability",
"Opportunities": "Explore new attention mechanisms, improve efficiency",
"datatset": "WMT 2014 English-to-German translation task"
},
{
"paper_name": "Exploring Hierarchical Structures for Recommender Systems",
"Focus Area of the paper": "Hierarchical Structures in Recommender Systems",
"Date": "01/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed hierarchical attention network for recommendation",
"Application": "Recommending products, services, content",
"Challenges": "Data sparsity, cold-start problem",
"Opportunities": "Explore new hierarchical structures, improve personalization",
"datatset": "MovieLens 100K, Amazon Reviews"
},
{
"paper_name": "Heterogeneous Graph Attention Network",
"Focus Area of the paper": "Heterogeneous Graph Attention Networks",
"Date": "08/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed HGAN for heterogeneous graph learning",
"Application": "Node classification, link prediction",
"Challenges": "Handling different node types and relations",
"Opportunities": "Explore new attention mechanisms for heterogeneous graphs",
"datatset": "Amazon co-purchasing network, DBLP citation network"
},
{
"paper_name": "Hyperbolic Graph Attention Network",
"Focus Area of the paper": "Hyperbolic Graph Attention Networks",
"Date": "02/2020",
"Methodology": "Quantitative",
"Key Findings": "Proposed HGAT for learning on hyperbolic graphs",
"Application": "Node classification, link prediction",
"Challenges": "Handling non-Euclidean space, model complexity",
"Opportunities": "Explore new hyperbolic embedding techniques",
"datatset": "Cora, PubMed, Amazon"
},
{
"paper_name": "KGAT: Knowledge Graph Attention Network for Recommendation",
"Focus Area of the paper": "Knowledge Graph Attention Networks for Recommendation",
"Date": "06/2019",
"Methodology": "Quantitative",
"Key Findings": "Proposed KGAT for incorporating knowledge graphs into recommendation",
"Application": "Recommending products, services, content",
"Challenges": "Scalability, knowledge graph incompleteness",
"Opportunities": "Explore new ways to integrate knowledge graphs",
"datatset": "Amazon, MovieLens"
},
{
"paper_name": "Knowledge Graph Embedding Based on Graph Neural Network",
"Focus Area of the paper": "Knowledge Graph Embedding using GNNs",
"Date": "03/2020",
"Methodology": "Quantitative",
"Key Findings": "Proposed GNN-based embedding for knowledge graphs",
"Application": "Knowledge graph completion, question answering",
"Challenges": "Data sparsity, model complexity",
"Opportunities": "Explore new GNN architectures for knowledge graphs",
"datatset": "FB15k-237, WN18RR"
},
{
"paper_name": "Research on the application of Nerual Network model in knowledge graph completion technology",
"Focus Area of the paper": "Knowledge Graph Completion using Neural Networks",
"Date": "06/2020",
"Methodology": "Review",
"Key Findings": "Surveyed NN models for knowledge graph completion",
"Application": "Knowledge graph completion, question answering",
"Challenges": "Data sparsity, model complexity",
"Opportunities": "Explore new NN architectures for knowledge graphs",
"datatset": "None"
},
{
"paper_name": "Attention based graph neural networks: a survey",
"Focus Area of the paper": "Attention Mechanisms in Graph Neural Networks",
"Date": "06/2021",
"Methodology": "Review",
"Key Findings": "Surveyed attention-based GNNs",
"Application": "Node classification, link prediction, graph classification",
"Challenges": "Computational cost, interpretability",
"Opportunities": "Explore new attention mechanisms, improve efficiency",
"datatset": "None"
},
{
"paper_name": "Graph neural networks for visual question answering: a systematic review",
"Focus Area of the paper": "Graph Neural Networks for Visual Question Answering",
"Date": "08/2021",
"Methodology": "Review",
"Key Findings": "Surveyed GNNs for visual question answering",
"Application": "Visual question answering, image captioning",
"Challenges": "Data scarcity, model complexity",
"Opportunities": "Explore new GNN architectures for visual tasks",
"datatset": "None"
},
{
"paper_name": "A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions",
"Focus Area of the paper": "Graph Neural Networks",
"Date": "12/2020",
"Methodology": "Review",
"Key Findings": "Comprehensive review of GNNs",
"Application": "Node classification, link prediction, graph classification",
"Challenges": "Scalability, interpretability, data sparsity",
"Opportunities": "Explore new GNN architectures, improve interpretability",
"datatset": "None"
}
]
```

# Final Automated Generation

A cleaner approach to run one by one: 
- will help with limited context
- possibly better accuracy and generation

In [6]:
system_promp_template = ("""
You are a [Research Assitant] bot. You help with creating [Literature Review]. 

Input :You will be given access to a [research paper]


Task: You have to [extract] the following information:
Information To BE extracted is present in backticks: 

Paper Name
Focus Area of the paper
Date: month/year
Methodology: (eg Qualitative, Quantitative, Review, Conceptual, Report)
Key Findings: in 10 words what the paper has implemented/achieved
Application: real life potential use cases (summerize in 10 words or 1-2 points)
Challenges:  Drawbacks of this paper/approach (summerize 1-2 short points)
Opportunities: Future scope/possibilities of paper (summerize in 1-2 points)
Dataset: (dataset used in the paper if available else write none) 

Note: if including multiple points, insert newline character or write a unordered list in markdown wrapped as a string

Output Format is inside triple backticks: 
```
json      
{
"paper_name": "name of the paper 0",
.
.
.
"opportunities": "..."
"datatset": "name of the dataset"
}
```
              
""")

In [7]:
user_query_template = """
{system_prompt_template}
Generate a literature review based on this research paper
{paper_name}
"""

In [8]:
from langchain.prompts import PromptTemplate

final_prompt_template= PromptTemplate.from_template(user_query_template)

**All our processing for the paper consolidated into one function** 

In [9]:
def process_paper(file_path, paper_name: str): 
    loader = PyPDFLoader(file_path)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap=200)
    texts = text_splitter.split_documents(data)
    
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
    vector_store = Chroma.from_documents(documents=texts, embedding=embeddings)

    retriever = vector_store.as_retriever()
    
    llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", 
        retriever=retriever,
        return_source_documents=True    
    )


    final_prompt = final_prompt_template.format(
        system_prompt_template=system_promp_template, 
        paper_name =paper_name
    )
    result = qa_chain.invoke({'query': final_prompt})
    
    return result
    

- A simple functions to parseout the JSON
- Alternatively you can setup JSONOutputParser from langchain.outputparsers (reqires additional setup)

In [10]:
import json
def extract_and_append_json(json_string):
    # Remove the ```json and ``` markers if present
    json_string = json_string.strip()
    if json_string.startswith("```json"):
        json_string = json_string[7:]
    if json_string.endswith("```"):
        json_string = json_string[:-3]
    
    # Parse the JSON string
    try:
        json_data = json.loads(json_string)
        
        # Append the parsed JSON to the result list
        return json_data

    except json.JSONDecodeError as e:
        print(e)

Name of all the papers you're working with

In [11]:

paper_names = [
"Self-Supervised Learning of Graph Neural Networks",
"""A Survey on Graph Neural Networks for Time
Series: Forecasting, Classification, Imputation,
and Anomaly Detection""", 
"""Attention Is All You Need""",
"""Exploring Hierarchical Structures for Recommender Systems""",
"""Heterogeneous Graph Attention Network""", 
"""Hyperbolic Graph Attention Network""", 
"""KGAT: Knowledge Graph Attention Network for Recommendation""", 
"""Knowledge Graph Embedding Based on Graph Neural Network""",
"""Research on the application of Nerual Network model in knowledge graph completion technology""",
"""Attention based graph neural networks: a survey""", 
"""Graph neural networks for visual question answering: a systematic review""", 
"""A review of graph neural networks: concepts, architectures, techniques, challenges, datasets, applications, and future directions"""
]


Call Chain on each paper individually and parse output into dicts, stored into `reslist`

In [13]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
papers_directory = './papers'

#stores the result
reslist = []
ct = 0

for filename in os.listdir(papers_directory):
    if ct > 9: break
    if ct == 2: pass
    if filename.endswith(".pdf"):
        file_path = os.path.join(papers_directory, filename)
        print(f"Processing: {file_path}")
        num = int(filename[:-4])


        result = process_paper(file_path, paper_names[ct])

        print(f"Processed Paper: {paper_names[ct]}")

        reslist.append(extract_and_append_json(result['result']))
    ct+=1

reslist

```Accessing One Object```

In [17]:
reslist[1]

{'paper_name': 'A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection',
 'focus_area_of_the_paper': 'Graph Neural Networks for Time Series Analysis',
 'date': '02/2024',
 'methodology': 'Review',
 'key_findings': 'Surveyed GNN applications for time series tasks',
 'application': '- Forecasting future values\n- Detecting anomalies in time series data',
 'challenges': '- Limited availability of suitable time series datasets\n- Scalability of GNNs for large-scale time series data',
 'opportunities': '- Development of new GNN architectures for time series\n- Integration of GNNs with other time series methods',
 'dataset': 'None'}

In [16]:
Markdown(reslist[1]['opportunities'])

- Development of new GNN architectures for time series
- Integration of GNNs with other time series methods

### We can also create another chain that takes in these json lists and turn those into MD

Why not then just return MD instead of JSON in first place? 
- allows to process each paper individually
- can feed into other chains easily
- cleaner and structured approach


In [18]:
create_markdown_tempalte = """
Given a list of JSON objects inside triple backticks. \
Generate a table in [Markdown] using the keys as Table Columns

```{list_of_papers}```
"""

md_prompt = PromptTemplate.from_template(create_markdown_tempalte)

fin_prompt = md_prompt.format(
    list_of_papers=reslist
)

In [19]:
llm = llm 

In [20]:
lit_review_md = llm.invoke(fin_prompt)

### Visualize: 

In [21]:
Markdown(lit_review_md)

| paper_name | focus_area_of_the_paper | date | methodology | key_findings | application | challenges | opportunities | dataset |
|---|---|---|---|---|---|---|---|---|
| Self-Supervised Learning of Graph Neural Networks | Graph Neural Networks, Self-Supervised Learning | 01/2021 | Review | Survey of self-supervised learning for GNNs | Drug discovery, social network analysis | Lack of standardized evaluation metrics, data scarcity | Development of new self-supervised learning methods, application to real-world problems | None |
| A Survey on Graph Neural Networks for Time Series: Forecasting, Classification, Imputation, and Anomaly Detection | Graph Neural Networks for Time Series Analysis | 02/2024 | Review | Surveyed GNN applications for time series tasks | - Forecasting future values<br>- Detecting anomalies in time series data | - Limited availability of suitable time series datasets<br>- Scalability of GNNs for large-scale time series data | - Development of new GNN architectures for time series<br>- Integration of GNNs with other time series methods | None |
| Attention Is All You Need | Machine Translation | 6/2017 | Quantitative | Proposes Transformer architecture for machine translation | Machine translation, text summarization, question answering | Computational complexity for long sequences, lack of inductive bias | Improved performance on various NLP tasks, further research on architecture variations | WMT 2014 English-to-German translation task |
| Exploring Hierarchical Structures for Recommender Systems | Recommender Systems | 12/2019 | Review | Survey of hierarchical structures in recommender systems | Improve recommendation accuracy and efficiency | Complexity of hierarchical structures, lack of standardized evaluation metrics | Develop novel hierarchical structures, explore applications in diverse domains | None |
| Heterogeneous Graph Attention Network | Graph Neural Networks, Heterogeneous Graphs | 12/2020 | Quantitative | Proposes a novel heterogeneous graph attention network | Recommender systems, knowledge graph completion | Scalability to large graphs,  handling complex graph structures | Further research on different attention mechanisms,  exploration of new applications | None |
| Hyperbolic Graph Attention Network | Graph Neural Networks | 12/2020 | Quantitative | Hyperbolic space improves graph attention networks | Node classification, link prediction, graph clustering | Hyperbolic space computation complexity,  Limited scalability | Explore applications in other domains,  Improve efficiency and scalability | Cora, Citeseer, PubMed |
| KGAT: Knowledge Graph Attention Network for Recommendation | Recommender Systems | 00/2020 | Quantitative | Knowledge graph attention network for recommendation | Personalized recommendations based on user preferences and item relationships | Overlooking relations among instances or items | Further exploration of different knowledge graph embedding techniques and attention mechanisms | Amazon-Book, Last-FM, Yelp2018 |
| Knowledge Graph Embedding Based on Graph Neural Network | Knowledge Graph Completion | N/A | Conceptual | Proposes a new approach for KG embedding | Improve knowledge representation and reasoning | Scalability for large knowledge graphs | Integration with other machine learning techniques | None |
| Attention-based graph neural networks: a survey | Attention-based Graph Neural Networks | 01/2023 | Review | Survey of attention-based GNN architectures | Graph-based machine learning tasks | Overfitting to specific graph structures<br>Limited scalability to large graphs | Development of more efficient and robust attention mechanisms<br>Exploration of novel applications in various domains | None |

Done

In [ ]:
print(lit_review_md)
display(Markdown(lit_review_md))


# Synopsis

# IDEA GENERATION - generate ideas for Paper

In [23]:
suggest_ideas_tempalte = """
You are a `Machine Learning` professor and we your students need to [work on final year project]. 
For that we have to come up with a list of papers to refer to. 

`Based on them we have to come up with some research gaps we can fulfil in certain domain or topic` with the goal of: 
`Write a Research Paper`
and `Create the final project`. 

We need your help to come up with <potential areas> our team can address as final year B.E. students. 

The project has to build upon existing research and its helpful if we create something that is `trending` in research like LLMs and VLM etc. 

Our areas of Interest: 
- Knowledge Representation (very important)
- GAT
- Deep Learning 
- LLMs



`our idea has to be novel`


END GOAL: 
<
- Come up with 3-5`objectives` that we can have as outcomes of our project
- final projects that we can do (END TO END Ideas)
- workflow or project plan for that project 
- scope of project should be 3 months
- expose us to a wide range of experiences
- help come up with names for the project 
>


Given a list of papers as `Literature Review` in Markdown format.

```{lit_review}```


Output Format: 
```markdown
# Project Title: Title of Project
# Paper Title: Title of Paper
# Abstract: 5-10 sentences
# Motivation: Why do we want to build on this research (4 bullet points)
# Objectives: <IDEAS that we are introducing newly to the field> (3-4 bullet points) <- should contain `keywords`
# Architecture: (you can't draw just tell the steps or names of them)
# Application Areas: `3-5` areas where our research can have an impact
# conclusion: 
# Project Plan: 
```


constraints: 
```
- It should not be too hard to develop for beginners 
- Should have a small scope not requiring years of scope
- At the end explain what all topics we need to study within 3 weeks to be knoledge-ible enough to make this project. 
```


COME UP WITH ONLY ONE IDEA
"""

md_prompt = PromptTemplate.from_template(suggest_ideas_tempalte)

synopsis_format = md_prompt.format(
    lit_review=lit_review_md
)

A high tempreature means more creative ideas 

In [24]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.7)

In [27]:
synopsis = llm.invoke(synopsis_format)

In [28]:
Markdown(synopsis)

## Project Title:  KG-LLM: Knowledge Graph Enhanced Large Language Models for Improved Reasoning and Question Answering

## Paper Title: KGAT: Knowledge Graph Attention Network for Recommendation

## Abstract: 

This project explores the integration of knowledge graphs (KGs) with large language models (LLMs) to enhance their reasoning capabilities and improve performance in question answering tasks. We aim to leverage the structured information in KGs to provide LLMs with a richer context and improved understanding of relationships between entities, leading to more accurate and nuanced answers. This approach builds upon existing research on KGAT (Knowledge Graph Attention Network) for recommendation systems and leverages the power of LLMs for natural language processing tasks.

## Motivation:

* **Limited Reasoning Capabilities of LLMs:** Current LLMs struggle with complex reasoning tasks, especially when dealing with factual knowledge or understanding intricate relationships between entities.
* **Knowledge Graph Potential:** KGs offer a structured representation of factual knowledge, providing LLMs with a valuable source of information to enhance their reasoning abilities.
* **Improving Question Answering:** Integrating KGs into LLMs can significantly improve question answering performance by providing a more comprehensive context and enabling the model to infer relationships between entities.
* **Real-world Applications:** Enhanced question answering systems with improved reasoning capabilities have applications in various domains such as customer service, information retrieval, and education.

## Objectives:

* **KG-LLM Architecture:**  Develop a novel architecture that effectively integrates a knowledge graph into the structure of a pre-trained LLM.
* **Knowledge-Guided Reasoning:** Implement mechanisms within the KG-LLM that enable the model to leverage KG information for improved reasoning and inference.
* **Question Answering Evaluation:** Evaluate the performance of the KG-LLM on benchmark question answering datasets, comparing it to traditional LLMs and other KG-based approaches.
* **Explainable Reasoning:**  Explore techniques to provide explanations for the reasoning process of the KG-LLM, enhancing transparency and trust in its predictions.

## Architecture:

The proposed KG-LLM architecture will consist of:

1. **Knowledge Graph Embedding:**  Embed the knowledge graph into a vector space using techniques like TransE or RotatE.
2. **LLM Integration:** Integrate the embedded KG into the pre-trained LLM, potentially through attention mechanisms or by fine-tuning the LLM on KG-related tasks.
3. **Reasoning Mechanism:** Develop a mechanism within the KG-LLM that enables the model to utilize the embedded KG for inference and reasoning during question answering.

## Application Areas:

* **Customer Service Chatbots:**  Enhance chatbot responses with factual knowledge and improved reasoning capabilities.
* **Information Retrieval Systems:**  Improve search results by leveraging KG information to understand user queries more effectively.
* **Educational Tools:**  Develop intelligent tutoring systems that can provide explanations and answer student questions more accurately.
* **Medical Diagnosis and Treatment:**  Assist medical professionals by providing insights based on medical knowledge graphs and patient data.

## Conclusion:

This project aims to address the limitations of traditional LLMs by integrating knowledge graphs to enhance their reasoning capabilities and improve question answering performance. The proposed KG-LLM architecture has the potential to significantly impact various domains by providing more accurate and insightful answers, leading to improved decision-making and better user experiences.

## Project Plan:

**Phase 1 (Week 1-3):**

* **Literature Review:**  Thorough study of existing research on KGAT, LLMs, and knowledge-based reasoning.
* **Data Acquisition:**  Select and prepare suitable knowledge graphs and question answering datasets.
* **Model Selection:**  Choose an appropriate LLM and KG embedding technique.
* **Architecture Design:**  Develop the initial design for the KG-LLM architecture.

**Phase 2 (Week 4-6):**

* **Model Implementation:**  Implement the KG-LLM architecture using deep learning frameworks.
* **Training and Evaluation:**  Train the model on the chosen datasets and evaluate its performance on various question answering tasks.
* **Analysis and Optimization:**  Analyze the model's performance and identify areas for improvement.
* **Explainability Techniques:**  Explore and implement techniques for explaining the KG-LLM's reasoning process.

**Phase 3 (Week 7-9):**

* **Final Evaluation and Reporting:**  Conduct a comprehensive evaluation of the KG-LLM's performance and document the findings in a research paper.
* **Presentation and Demonstration:**  Prepare a presentation and demonstration of the project for final evaluation.
* **Project Documentation:**  Create a detailed report documenting the project's methodology, results, and future directions.

## Topics to Study:

* **Knowledge Graph Embeddings:** Techniques like TransE, RotatE, ComplEx, etc.
* **Large Language Models:**  Architecture, pre-training, fine-tuning, and applications.
* **Attention Mechanisms:**  Self-attention, multi-head attention, and their role in LLMs.
* **Question Answering Techniques:**  Different approaches to question answering, including retrieval-based, generative, and knowledge-based methods.
* **Deep Learning Frameworks:**  TensorFlow, PyTorch, or other suitable frameworks for implementing the KG-LLM.

This project provides a challenging and rewarding experience for final year B.E. students, exposing them to cutting-edge research in LLMs and knowledge graph integration. By successfully integrating a knowledge graph into an LLM, students will contribute to the advancement of artificial intelligence and gain valuable experience in a rapidly growing field. 
